<div style="text-align: center;"><div  style="color:#03bafc; font-size:30px; font-weight:bold; line-height:40px;">Tweepy - Tweet Extraction</div></div>
<div style="text-align: center; color:#006666"><strong>Created By: </strong>Parikshit Koya</div>
<br>
<div style="text-align: center; color:#006666"><strong>GitHub: </strong><a href='https://github.com/parikshitkoya'>https://github.com/parikshitkoya</a></div>
<div style="text-align: center; color:#006666"><strong>Kaggle: </strong><a href='https://www.kaggle.com/parikshitkoya'>https://www.kaggle.com/parikshitkoya</a></div>


#### What are we doing:

- Downloading 3000 Tweets for where a twitter handle has been mentioned.
- Extracting specific columns by parsing the JSON (JavaScript Object Notation) output from Tweepy
- Extracting full tweets using the tweet ID as tweets are Truncated

In [71]:
from secrets import apikey, apikeysecret, accesstoken, accesstokensecret

### Importing the required libraries

In [73]:
import tweepy
from tweepy import Cursor
import pandas as pd
import json
import openpyxl
from datetime import datetime, timedelta
import re
import string
import matplotlib.pyplot as plt

import numpy as np
import nltk
import sklearn as sk
from sklearn.metrics import precision_recall_fscore_support as score


In [74]:
auth = tweepy.OAuthHandler(apikey, apikeysecret)
auth.set_access_token(accesstoken, accesstokensecret)
api = tweepy.API(auth, wait_on_rate_limit=True)

#### Importing Tweets from @AmazonIN

In [101]:
query = '@amazonin -filter:retweets'
max_tweets = 4000
searched_tweets_amazon = [status for status in tweepy.Cursor(api.search, lang = 'en', q=query).items(max_tweets)]
tweet_id = []
text = []
source = []
user = []
created = []
user_verified = []
for i in range(len(searched_tweets_amazon)):
    json_str = json.dumps(searched_tweets_amazon[i]._json)
    json.loads(json_str)['id']
    tweet_id.append(str(json.loads(json_str)['id']))
    text.append(json.loads(json_str)['text'])
    source.append(json.loads(json_str)['source'])
    user.append(json.loads(json_str)['user']['screen_name'])
    created.append(json.loads(json_str)['created_at'])
    user_verified.append(json.loads(json_str)['user']['verified'])

In [102]:
amzn_dict = {'tweet_id': tweet_id, 'text': text, 'user': user, 'created': created, 'user_verified': user_verified}
amzn_tweets = pd.DataFrame(amzn_dict)
amzn_tweets

,tweet_id,text,user,created,user_verified
0,1351150534800117760,@amazonIN @AmazonHelp what's wrong??? After 6 ...,YashDaga,Mon Jan 18 12:52:45 +0000 2021,False
1,1351150515703455752,@amazonIN A .3\nB. 2\nC. 1\nD. 4,Chandan19408317,Mon Jan 18 12:52:40 +0000 2021,False
2,1351150469343809543,"@amazonIN A) REDMI 9A - 3) Rs. 6,999\nB) Samsu...",MollaNew,Mon Jan 18 12:52:29 +0000 2021,False
3,1351150342893948931,"@amazonIN A. Redmi 9A - 3. Rs 6,999🔥\nB. Samsu...",MadhuChand07,Mon Jan 18 12:51:59 +0000 2021,False
4,1351150232634064901,Aura by Samyak Naturals is now Amazon's Choice...,SamyakNaturals,Mon Jan 18 12:51:33 +0000 2021,False
...,...,...,...,...,...
3995,1350771816193810433,@amazonIN 19 th January #AmazonPrimeMembership...,Royal__Jadeja,Sun Jan 17 11:47:51 +0000 2021,False
3996,1350771771130085378,"@amazonIN 19th January , 2021\n\n#AmazonPrimeM...",Param_Sidhu77,Sun Jan 17 11:47:41 +0000 2021,False
3997,1350771761063759873,@amazonIN 26th january,Pradhanadityaj1,Sun Jan 17 11:47:38 +0000 2021,False
3998,1350771684282810373,@RNagothu @PrakashJavdekar @KapilMishra_IND @R...,giamvt,Sun Jan 17 11:47:20 +0000 2021,False


In [105]:
full_text = []
for i in range(len(amzn_tweets)):
    try:
        full_text.append(api.get_status(amzn_tweets.iloc[i][0], tweet_mode='extended')._json['full_text'])
    except:
        full_text.append(amzn_tweets.iloc[i][1])
amzn_tweets['full_text'] = full_text
amzn_tweets

,tweet_id,text,user,created,user_verified,full_text
0,1351150534800117760,@amazonIN @AmazonHelp what's wrong??? After 6 ...,YashDaga,Mon Jan 18 12:52:45 +0000 2021,False,@amazonIN @AmazonHelp what's wrong??? After 6 ...
1,1351150515703455752,@amazonIN A .3\nB. 2\nC. 1\nD. 4,Chandan19408317,Mon Jan 18 12:52:40 +0000 2021,False,@amazonIN A .3\nB. 2\nC. 1\nD. 4
2,1351150469343809543,"@amazonIN A) REDMI 9A - 3) Rs. 6,999\nB) Samsu...",MollaNew,Mon Jan 18 12:52:29 +0000 2021,False,"@amazonIN A) REDMI 9A - 3) Rs. 6,999\nB) Samsu..."
3,1351150342893948931,"@amazonIN A. Redmi 9A - 3. Rs 6,999🔥\nB. Samsu...",MadhuChand07,Mon Jan 18 12:51:59 +0000 2021,False,"@amazonIN A. Redmi 9A - 3. Rs 6,999🔥\nB. Samsu..."
4,1351150232634064901,Aura by Samyak Naturals is now Amazon's Choice...,SamyakNaturals,Mon Jan 18 12:51:33 +0000 2021,False,Aura by Samyak Naturals is now Amazon's Choice...
...,...,...,...,...,...,...
3995,1350771816193810433,@amazonIN 19 th January #AmazonPrimeMembership...,Royal__Jadeja,Sun Jan 17 11:47:51 +0000 2021,False,@amazonIN 19 th January #AmazonPrimeMembership...
3996,1350771771130085378,"@amazonIN 19th January , 2021\n\n#AmazonPrimeM...",Param_Sidhu77,Sun Jan 17 11:47:41 +0000 2021,False,"@amazonIN 19th January , 2021\n\n#AmazonPrimeM..."
3997,1350771761063759873,@amazonIN 26th january,Pradhanadityaj1,Sun Jan 17 11:47:38 +0000 2021,False,@amazonIN 26th january
3998,1350771684282810373,@RNagothu @PrakashJavdekar @KapilMishra_IND @R...,giamvt,Sun Jan 17 11:47:20 +0000 2021,False,@RNagothu @PrakashJavdekar @KapilMishra_IND @R...


In [106]:
amzn_tweets.to_excel('amazon_tweets.xlsx')

In [27]:
auth = tweepy.OAuthHandler(apikey, apikeysecret)
auth.set_access_token(accesstoken, accesstokensecret)
api = tweepy.API(auth, wait_on_rate_limit=True)

#### Importing Tweets from @Flipkart

In [92]:
query = '@flipkart -filter:retweets'
max_tweets = 4000
searched_tweets_flipkart = [status for status in tweepy.Cursor(api.search, lang = 'en', q=query).items(max_tweets)]
tweet_id = []
text = []
user = []
created = []
user_verified = []
for i in range(len(searched_tweets_flipkart)):
    json_str = json.dumps(searched_tweets_flipkart[i]._json)
    json.loads(json_str)['id']
    tweet_id.append(str(json.loads(json_str)['id']))
    text.append(json.loads(json_str)['text'])
    user.append(json.loads(json_str)['user']['screen_name'])
    created.append(json.loads(json_str)['created_at'])
    user_verified.append(json.loads(json_str)['user']['verified'])

In [93]:
flipkart_dict = {'tweet_id': tweet_id, 'text': text, 'user': user, 'created': created, 'user_verified': user_verified}
flipkart_tweets = pd.DataFrame(flipkart_dict)
flipkart_tweets

,tweet_id,text,user,created,user_verified
0,1351081210743820289,@Euro_Fashions @amazonIN @Flipkart Fake contes...,Ramprasad43,Mon Jan 18 08:17:17 +0000 2021,False
1,1351080341906333698,"The #OPPOReno5Pro 5G priced at Rs. 35,990 and ...",waquarhaider10,Mon Jan 18 08:13:50 +0000 2021,False
2,1351080322461536258,"@flipkartsupport Even if you won't accept, eve...",JayashreeKawad2,Mon Jan 18 08:13:45 +0000 2021,False
3,1351079901173051392,@Flipkart Mera ek product 11th jan ko pickup h...,ruturaj_art,Mon Jan 18 08:12:04 +0000 2021,False
4,1351079541733797890,@oppomobileindia EMI Months Rs. 1000\n@Flipkart,IslamNabiAnsari,Mon Jan 18 08:10:39 +0000 2021,False
...,...,...,...,...,...
3304,1350703146494877699,@IndiaPOCO @NatwarlalMe @Flipkart what about #...,Laxmipr44761376,Sun Jan 17 07:14:59 +0000 2021,False
3305,1350702989753806849,@Flipkart What the hell is going on ??\nWhy ar...,yash_kushwah_,Sun Jan 17 07:14:22 +0000 2021,False
3306,1350702947987058688,@flipkartsupport Fake @Flipkart,GoharTasha,Sun Jan 17 07:14:12 +0000 2021,False
3307,1350702895759384576,@Mayankujjaini @Flipkart heard about display i...,majestic_minal,Sun Jan 17 07:13:59 +0000 2021,False


In [99]:
full_text = []
for i in range(len(flipkart_tweets)):
    try:
        full_text.append(api.get_status(flipkart_tweets.iloc[i][0], tweet_mode='extended')._json['full_text'])
    except:
        full_text.append(flipkart_tweets.iloc[i][1])
flipkart_tweets['full_text'] = full_text
flipkart_tweets

,tweet_id,text,user,created,user_verified,full_text
0,1351081210743820289,@Euro_Fashions @amazonIN @Flipkart Fake contes...,Ramprasad43,Mon Jan 18 08:17:17 +0000 2021,False,@Euro_Fashions @amazonIN @Flipkart Fake contes...
1,1351080341906333698,"The #OPPOReno5Pro 5G priced at Rs. 35,990 and ...",waquarhaider10,Mon Jan 18 08:13:50 +0000 2021,False,"The #OPPOReno5Pro 5G priced at Rs. 35,990 and ..."
2,1351080322461536258,"@flipkartsupport Even if you won't accept, eve...",JayashreeKawad2,Mon Jan 18 08:13:45 +0000 2021,False,"@flipkartsupport Even if you won't accept, eve..."
3,1351079901173051392,@Flipkart Mera ek product 11th jan ko pickup h...,ruturaj_art,Mon Jan 18 08:12:04 +0000 2021,False,@Flipkart Mera ek product 11th jan ko pickup h...
4,1351079541733797890,@oppomobileindia EMI Months Rs. 1000\n@Flipkart,IslamNabiAnsari,Mon Jan 18 08:10:39 +0000 2021,False,@oppomobileindia EMI Months Rs. 1000\n@Flipkart
...,...,...,...,...,...,...
3304,1350703146494877699,@IndiaPOCO @NatwarlalMe @Flipkart what about #...,Laxmipr44761376,Sun Jan 17 07:14:59 +0000 2021,False,@IndiaPOCO @NatwarlalMe @Flipkart what about #...
3305,1350702989753806849,@Flipkart What the hell is going on ??\nWhy ar...,yash_kushwah_,Sun Jan 17 07:14:22 +0000 2021,False,@Flipkart What the hell is going on ??\nWhy ar...
3306,1350702947987058688,@flipkartsupport Fake @Flipkart,GoharTasha,Sun Jan 17 07:14:12 +0000 2021,False,@flipkartsupport Fake @Flipkart
3307,1350702895759384576,@Mayankujjaini @Flipkart heard about display i...,majestic_minal,Sun Jan 17 07:13:59 +0000 2021,False,@Mayankujjaini @Flipkart heard about display i...


In [100]:
flipkart_tweets.to_excel('flipkart_tweets.xlsx')

#### Importing Tweets from @Snapdeal

Attempting to download 3000 Tweets with a different set of Credentials:

In [84]:
def convtodatetime(text):
    text = datetime.strptime(text, '%a %b %d %H:%M:%S %z %Y')
    return text

In [85]:
starttime = datetime.utcnow()
starttime = starttime - timedelta(hours=0, minutes=0, seconds = 15)
startdate = starttime.strftime('%Y%m%d%H%M')
print(startdate)

202101180704


In [86]:
query = '@snapdeal'
tweet_id = []
text = []
user = []
created = []
user_verified = []

while len(tweet_id)<4000:
    searched_tweets_snapdeal = [status for status in api.search_full_archive(environment_name='TestEnv', query=query, toDate = startdate, maxResults = 100)]

    for i in range(len(searched_tweets_snapdeal)):
        json_str = json.dumps(searched_tweets_snapdeal[i]._json)
        tweet_id.append(str(json.loads(json_str)['id']))
        text.append(json.loads(json_str)['text'])
        user.append(json.loads(json_str)['user']['screen_name'])
        created.append(json.loads(json_str)['created_at'])
        user_verified.append(json.loads(json_str)['user']['verified'])

    temp_timestamp = pd.DataFrame(created ,columns=['temp_time'])
    temp_timestamp['temp_time'] = temp_timestamp['temp_time'].apply(convtodatetime)
    startdate = min(temp_timestamp['temp_time'])
    startdate = startdate.strftime('%Y%m%d%H%M')
    startdate

Wanted to download 4000 tweets but was able to download only 3000 due to API restrictions

In [87]:
snapdeal_dict = {'tweet_id': tweet_id, 'text': text, 'user': user, 'created': created, 'user_verified': user_verified}
snapdeal_tweets = pd.DataFrame(snapdeal_dict)
snapdeal_tweets

,tweet_id,text,user,created,user_verified
0,1351027592799932424,It's a fraud or I really win price from snapde...,Deshbha32859266,Mon Jan 18 04:44:13 +0000 2021,False
1,1351027067803070467,#फ़सवनुक\n@Flipkart हे ग्राहकांची फ़सवनुक करत ...,SAGARANDHARE4,Mon Jan 18 04:42:08 +0000 2021,False
2,1351004828315561987,Dear @Flipkart ur courier partner is a lier. ...,Adhithy33736019,Mon Jan 18 03:13:46 +0000 2021,False
3,1350953560008175619,@snapdeal 91287989,Rakeshk92153278,Sun Jan 17 23:50:02 +0000 2021,False
4,1350865862547316736,RT @forbes_india: #2021LivingInHope As we stay...,PrachiSharma707,Sun Jan 17 18:01:34 +0000 2021,False
...,...,...,...,...,...
4089,1322228171173892096,Big billion stories 🤷‍♂️\n\nHey @Flipkart Why ...,trendtrader93,Fri Oct 30 17:25:36 +0000 2020,False
4090,1322221761300099072,@UniverseUNLTD @devgandhi1988 @Dilljaley @Megh...,Lucille_Is_Mine,Fri Oct 30 17:00:08 +0000 2020,False
4091,1322214169353990150,@Lucille_Is_Mine @Dilljaley @MeghUpdates @amaz...,devgandhi1988,Fri Oct 30 16:29:58 +0000 2020,False
4092,1322213873672355841,@devgandhi1988 @Dilljaley @MeghUpdates @amazon...,Lucille_Is_Mine,Fri Oct 30 16:28:47 +0000 2020,False


In [90]:
full_text = []
for i in range(len(snapdeal_tweets)):
    try:
        full_text.append(api.get_status(snapdeal_tweets.iloc[i][0], tweet_mode='extended')._json['full_text'])
    except:
        full_text.append(snapdeal_tweets2.iloc[i][0])
snapdeal_tweets['full_text'] = full_text
snapdeal_tweets

,tweet_id,text,user,created,user_verified,full_text
0,1351027592799932424,It's a fraud or I really win price from snapde...,Deshbha32859266,Mon Jan 18 04:44:13 +0000 2021,False,It's a fraud or I really win price from snapde...
1,1351027067803070467,#फ़सवनुक\n@Flipkart हे ग्राहकांची फ़सवनुक करत ...,SAGARANDHARE4,Mon Jan 18 04:42:08 +0000 2021,False,#फ़सवनुक\n@Flipkart हे ग्राहकांची फ़सवनुक करत ...
2,1351004828315561987,Dear @Flipkart ur courier partner is a lier. ...,Adhithy33736019,Mon Jan 18 03:13:46 +0000 2021,False,Dear @Flipkart ur courier partner is a lier. ...
3,1350953560008175619,@snapdeal 91287989,Rakeshk92153278,Sun Jan 17 23:50:02 +0000 2021,False,@snapdeal 91287989
4,1350865862547316736,RT @forbes_india: #2021LivingInHope As we stay...,PrachiSharma707,Sun Jan 17 18:01:34 +0000 2021,False,RT @forbes_india: #2021LivingInHope As we stay...
...,...,...,...,...,...,...
4089,1322228171173892096,Big billion stories 🤷‍♂️\n\nHey @Flipkart Why ...,trendtrader93,Fri Oct 30 17:25:36 +0000 2020,False,Big billion stories 🤷‍♂️\n\nHey @Flipkart Why ...
4090,1322221761300099072,@UniverseUNLTD @devgandhi1988 @Dilljaley @Megh...,Lucille_Is_Mine,Fri Oct 30 17:00:08 +0000 2020,False,@UniverseUNLTD @devgandhi1988 @Dilljaley @Megh...
4091,1322214169353990150,@Lucille_Is_Mine @Dilljaley @MeghUpdates @amaz...,devgandhi1988,Fri Oct 30 16:29:58 +0000 2020,False,@Lucille_Is_Mine @Dilljaley @MeghUpdates @amaz...
4092,1322213873672355841,@devgandhi1988 @Dilljaley @MeghUpdates @amazon...,Lucille_Is_Mine,Fri Oct 30 16:28:47 +0000 2020,False,@devgandhi1988 @Dilljaley @MeghUpdates @amazon...


In [91]:
snapdeal_tweets.to_excel('snapdeal_tweets.xlsx')